In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip uninstall nltk -y
!pip install texthero

Uninstalling nltk-3.2.5:
  Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 1.4MB 10.5MB/s 
     |████████████████████████████████| 245kB 34.0MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434677 sha256=02d56414729290c61d26d09f93734815f09e7f7da340e3b3b22059519f5f59fd
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [3]:
# %cd /content/drive/MyDrive/citation_prediction/
# !ls
# !git clone https://github.com/facebookresearch/fastText.git
# %cd ./fastText
# !make

In [4]:
import os

import numpy as np
import pandas as pd

import dill
from sklearn.feature_extraction.text import TfidfVectorizer
import gc
from tqdm import tqdm
import time

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import hashlib
from bs4 import BeautifulSoup
import nltk

import texthero as hero
from texthero import preprocessing

INPUT_DIR = '/content/drive/MyDrive/citation_prediction/input/'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


データの読み込み

In [5]:
all_title_abstract = pd.read_feather(os.path.join(INPUT_DIR, 'all_title_abstract_df.feather'))
all_title_abstract.head()

,title,abstract,cites,clean_title,clean_abstract
0,Heavy-Flavour Production at HERA,We review the theoretical and experimental s...,NaN,heavy flavour production hera,review theoretical experimental status hea...
1,Spectropolarimetric Constraints on the Nature ...,While it is well recognized that interstella...,7.0,spectropolarimetric constraints nature inte...,well recognized interstellar grains made ...
2,A joint analysis of Planck and BICEP2 B modes ...,We analyze BICEP2 and Planck data using a mo...,188.0,joint analysis planck bicep2 b modes includ...,analyze bicep2 planck data using model inc...
3,Molecular movie of ultrafast coherent rotation...,Recording molecular movies on ultrafast time...,8.0,molecular movie ultrafast coherent rotational...,recording molecular movies ultrafast timescal...
4,A Modified Mixed Domain Method for Modeling Ac...,"In this paper, phase correction and amplitud...",NaN,modified mixed domain method modeling acoust...,paper phase correction amplitude compensati...


In [6]:
start = time.time()

sentences = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(all_title_abstract['clean_title']):
    try:
        # Split a review into parsed sentences.
        result = review
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(h)
    except:
        continue

num_features = 200     # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 1       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
seed = 42

hashfxn = lambda x: int(hashlib.md5(str(x).encode()).hexdigest(), 16)

print ("Training FastText model...")
# Train FastText model.
model = FastText(sentences, workers=num_workers, hs = 0, sg = 1, negative = 10, iter = 25,\
            size=num_features, min_count = min_word_count, hashfxn=hashfxn,\
            window = context, sample = downsampling, seed=seed)

model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" \
+ str(context) + "context_len2alldata"
model.init_sims(replace=True)

endmodeltime = time.time()

print ("time : ", endmodeltime-start)

  4%|▎         | 33009/910608 [00:00<00:02, 330083.23it/s]

Parsing sentences from training set...


100%|██████████| 910608/910608 [00:03<00:00, 232414.98it/s]


Training FastText model...
time :  5336.079612731934


In [ ]:
# %cd ../input
# !ls

# label_wakati=[]
# for title in all_title_abstract['clean_title'].values:
#     label_wakati.append(title)
    
# f=open('title_list.txt',"w")
# for e in label_wakati:
#     f.write(e+"\n")
# f.close()

# %cd ../fastText
# !ls

# # 必要なものを諸々import
# import re, os
# from glob import glob
# from gensim.models.wrappers.fasttext import FastText

# # fastTextのコマンドを指定
# fasttext = "./fasttext"
# # アルゴリズム指定。skipgramにしとく
# algorithm = "skipgram"
# # fastTextで学習する分かち書きされたコーパスデータ
# input_file = "-input ../input/title_list.txt"
# # 分散表現のモデル名
# # この場合カレントディレクトリに「fasttext_title.model.bin」と「fasttext_title.model.vec」の２つのファイルができる
# output_model = "-output ../input/fasttext_title.model"

# # 諸々のパラメータを指定
# feature = "-dim 200"
# negative = "-neg 10"
# window_size = "-ws 5"
# epoch="-epoch 25"
# # fastTextコマンドをpythonから呼び出すために全部がっちゃんこする
# command_list = [fasttext, algorithm, input_file, output_model, feature, negative, window_size, epoch]
# command = " ".join(command_list)

# print("Learning for fasttext_title ...")
# os.system(command)
# print("Done.")

# # モデル読み込み
# # fastTextでモデルを読み込む時は .bin、.vecの指定はしないでロード
# model_name = "../input/fasttext_title.model"
# model = FastText.load_fasttext_format(model_name)

In [7]:
tqdm.pandas()

def des_to_mean_vec(text):
    
    try:
        text = text.strip()
        text_ls = [s for s in text.split(' ') if '' != s]
        return np.mean([model.wv[word] for word in text_ls if word in model.wv.index2word], axis=0)
    except:
        return np.nan

df = all_title_abstract['clean_title'].progress_apply(lambda x: des_to_mean_vec(x))
df.head()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning:

The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version

  0%|          | 904/910608 [00:00<7:47:25, 32.44it/s] /usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning:

Mean of empty slice.

100%|██████████| 910608/910608 [08:20<00:00, 1820.66it/s]


0    [0.016770303, -0.010412506, 0.03472478, 0.0951...
1    [-0.0028365864, 0.002412043, 0.0088761225, 0.0...
2    [-0.002221506, -0.04127803, -0.061291575, 0.06...
3    [-0.022794014, -0.02450148, 0.010705479, 0.118...
4    [-0.0017351928, -0.055854093, -0.006706296, 0....
Name: clean_title, dtype: object

In [8]:
title_df = df.apply(pd.Series)
title_df.columns = ['fasttext_title_' + str(col) for col in title_df.columns]
title_df.head()

,fasttext_title_0,fasttext_title_1,fasttext_title_2,fasttext_title_3,fasttext_title_4,fasttext_title_5,fasttext_title_6,fasttext_title_7,fasttext_title_8,fasttext_title_9,fasttext_title_10,fasttext_title_11,fasttext_title_12,fasttext_title_13,fasttext_title_14,fasttext_title_15,fasttext_title_16,fasttext_title_17,fasttext_title_18,fasttext_title_19,fasttext_title_20,fasttext_title_21,fasttext_title_22,fasttext_title_23,fasttext_title_24,fasttext_title_25,fasttext_title_26,fasttext_title_27,fasttext_title_28,fasttext_title_29,fasttext_title_30,fasttext_title_31,fasttext_title_32,fasttext_title_33,fasttext_title_34,fasttext_title_35,fasttext_title_36,fasttext_title_37,fasttext_title_38,fasttext_title_39,...,fasttext_title_160,fasttext_title_161,fasttext_title_162,fasttext_title_163,fasttext_title_164,fasttext_title_165,fasttext_title_166,fasttext_title_167,fasttext_title_168,fasttext_title_169,fasttext_title_170,fasttext_title_171,fasttext_title_172,fasttext_title_173,fasttext_title_174,fasttext_title_175,fasttext_title_176,fasttext_title_177,fasttext_title_178,fasttext_title_179,fasttext_title_180,fasttext_title_181,fasttext_title_182,fasttext_title_183,fasttext_title_184,fasttext_title_185,fasttext_title_186,fasttext_title_187,fasttext_title_188,fasttext_title_189,fasttext_title_190,fasttext_title_191,fasttext_title_192,fasttext_title_193,fasttext_title_194,fasttext_title_195,fasttext_title_196,fasttext_title_197,fasttext_title_198,fasttext_title_199
0,0.016770,-0.010413,0.034725,0.095134,0.000837,0.004979,0.007287,-0.002122,0.030440,0.042111,0.046247,-0.058751,0.067469,0.048763,0.019509,-0.034720,-0.048662,0.056322,-0.004835,0.038086,-0.074433,0.048588,0.065657,0.067029,-0.069154,-0.056876,0.115830,0.055118,0.102529,0.031608,-0.044598,0.013457,0.055204,0.041245,0.027926,-0.029444,-0.002071,0.029607,-0.014855,-0.003785,...,0.058870,0.020534,-0.013562,-0.103640,0.049168,0.026295,0.042327,0.060008,0.030616,0.037234,-0.044409,-0.009615,-0.047151,0.061650,0.024479,-0.029028,-0.022383,0.033602,0.049849,-0.041197,0.001570,0.061018,0.037548,0.018847,-0.028840,-0.033830,0.023398,-0.023147,0.063708,0.024194,0.009067,-0.025690,0.078976,-0.068683,-0.015801,-0.007297,0.091214,0.037782,-0.009763,-0.018836
1,-0.002837,0.002412,0.008876,0.069414,-0.087548,-0.085665,0.034693,0.066370,-0.040504,0.005440,0.053156,0.015926,0.028285,0.098147,-0.052698,-0.010871,-0.024291,0.020237,-0.044020,0.014535,-0.049495,-0.023361,0.059278,0.011039,-0.088248,-0.004104,-0.003587,0.059428,-0.034015,-0.028139,0.043865,-0.008554,-0.030129,0.083062,0.004465,0.008921,0.103865,-0.018011,0.004973,-0.027378,...,0.012464,-0.012437,-0.010231,-0.072426,0.005069,-0.005167,0.027583,0.043035,-0.000593,-0.045631,-0.020685,0.076627,-0.001460,0.027146,0.043702,-0.025485,0.049048,-0.016759,0.052759,-0.016585,-0.030680,0.028794,0.024825,-0.009279,0.042385,-0.013511,-0.022726,-0.007902,-0.037932,0.011110,-0.040958,-0.055733,0.059720,-0.030864,-0.043493,-0.032923,-0.042377,-0.008541,-0.056184,0.094780
2,-0.002222,-0.041278,-0.061292,0.060510,-0.044294,-0.031248,0.035476,0.025777,-0.012086,0.019486,-0.002919,0.009886,0.019346,0.031774,-0.025235,-0.059790,-0.048909,0.062787,-0.083599,0.015977,-0.065218,0.028079,0.029878,0.009921,-0.092581,-0.053489,0.044893,0.020905,-0.023619,-0.040942,-0.001941,0.043757,0.074863,0.023813,-0.034399,-0.034424,0.022898,0.054092,0.036896,-0.043881,...,0.090646,-0.021833,-0.013770,-0.055883,0.012173,-0.007850,0.007744,0.010013,-0.029687,-0.023958,0.028472,0.080436,-0.060951,0.006040,0.045729,-0.002499,0.027249,-0.002251,0.022776,-0.049531,-0.022536,0.017586,0.035942,-0.020164,0.000447,0.055539,0.009895,-0.009559,-0.079767,-0.021210,-0.054707,-0.027664,0.050146,-0.018119,0.019272,0.031946,0.012737,-0.033725,-0.012647,0.044040
3,-0.022794,-0.024501,0.010705,0.118241,-0.027668,-0.008393,-0.013427,0.043516,-0.058628,-0.008798,-0.010907,0.016781,0.053493,-0.011352,-0.075338,0.028389,-0.030398,0.012587,-0.056749,0.006836,-0.079743,0.007856,0.045248,0.021692,-

In [9]:
title_df.to_feather(os.path.join(INPUT_DIR, 'title_fasttext_df.feather'))

In [10]:
file = os.path.join(INPUT_DIR, 'fasttext_model_title.dill')
dill.dump(model, open(file,'wb'))